<a href="https://colab.research.google.com/github/Devasy23/FaceRec/blob/feat%2FModel-evaluation/Model-Training/Evaluate_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface

# Downloading Data

In [4]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2024-07-14 19:01:26--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M   308KB/s    in 6m 39s  

2024-07-14 19:08:06 (442 KB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [ ]:
!tar -xzvf lfw.tgz

In [6]:
import os
import shutil
count=0
for folder in os.listdir('lfw'):
    if len(os.listdir(f'lfw/{folder}')) < 5:
        shutil.rmtree(f'lfw/{folder}')
        count+=1
print(f"Removed {count} folders")
print(f"Remaining folders: {len(os.listdir('lfw'))}")

Removed 5326 folders


# Downloading Model

In [ ]:
!huggingface-cli login

In [ ]:
finetuned_model_dir = "model"

from huggingface_hub import snapshot_download
snapshot_download(repo_id='DnD11/FaceNet_Finetuned', repo_type="model", local_dir=finetuned_model_dir)

# Evaluation

In [ ]:
import os
import numpy as np
from tqdm import tqdm_notebook
from keras.models import load_model
from keras.preprocessing import image
from sklearn.metrics.pairwise import euclidean_distances
from deepface import DeepFace
import tensorflow as tf

# Function to load and preprocess images
def load_and_preprocess_image(img_path, target_size=(160, 160)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# Function to generate embeddings
def generate_embeddings(model, dataset_path, deepface = False):
    embeddings = []
    labels = []

    for class_name in tqdm_notebook(os.listdir(dataset_path)):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img_array = load_and_preprocess_image(img_path)
            if deepface:
                embedding = DeepFace.represent(img_path=img_path, model_name = "Facenet512", enforce_detection=False)
            else:
                embedding = model.predict(img_array)
            embeddings.append(embedding[0])
            labels.append(class_name)

    embeddings = np.array(embeddings)
    labels = np.array(labels)
    return embeddings, labels

# Function to calculate intra-cluster distances
def calculate_intra_cluster_distances(embeddings, labels):
    unique_labels = np.unique(labels)
    distances = []

    for label in unique_labels:
        cluster_embeddings = embeddings[labels == label]
        avg_embedding = np.mean(cluster_embeddings, axis=0)
        max_distance = np.max(euclidean_distances(cluster_embeddings, [avg_embedding]))
        distances.append(max_distance)

    return np.array(distances)

# Load the pre-trained FaceNet model (replace 'facenet_model.h5' with your model file)
# model_path = 'facenet_model.h5'
# model = load_model(model_path)

# Path to the dataset
dataset_path = 'lfw'
model=1
# Generate embeddings for the original model
embeddings_original, labels = generate_embeddings(model, dataset_path, True)

# Load the fine-tuned model 
finetuned_model = tf.saved_model.load(export_dir=finetuned_model_dir)
embeddings_finetuned, _ = generate_embeddings(finetuned_model, dataset_path)

# Calculate intra-cluster distances for both models
intra_distances_original = calculate_intra_cluster_distances(embeddings_original, labels)
intra_distances_finetuned = calculate_intra_cluster_distances(embeddings_finetuned, labels)

# Compare intra-cluster distances
intra_distance_change = intra_distances_finetuned - intra_distances_original

# Output the results
print(f"Intra-Cluster Distance Change: {intra_distance_change}")
print(f"Mean Distance Change: {np.mean(intra_distance_change)}")
print(f"Positive Impact: {np.sum(intra_distance_change < 0)}")
print(f"Negative Impact: {np.sum(intra_distance_change > 0)}")
print(f"Average Impact: {np.sum(intra_distance_change == 0)}")

  0%|          | 0/423 [00:00<?, ?it/s]

In [12]:
from deepface import DeepFace
embedding = DeepFace.represent(img_path='/content/lfw/Abdullah_Gul/Abdullah_Gul_0001.jpg', model_name = "Facenet512")

24-07-14 19:10:52 - Directory /root/.deepface created
24-07-14 19:10:52 - Directory /root/.deepface/weights created
24-07-14 19:10:56 - facenet512_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5
100%|██████████| 95.0M/95.0M [00:00<00:00, 199MB/s]


In [2]:
intra_distances_original.shape

NameError: name 'intra_distances_original' is not defined